In [48]:
## Imports
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import nltk
import datasketch
import tqdm
import pytest
import time
import pickle

In [49]:
df_patient = pd.read_csv("db_clean.csv")

In [50]:
df_patient.head()

,Unnamed: 0,patient_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,state_corrected,date_of_birth,dob_date,age,age_inferred,test_year,phone_number,phone_format,phone_geo
0,0,221958,matisse,clarke,13,rene street,westella,ellenbrook,2527,wa,wa,19710708.0,1971-07-08,32.0,32.0,2003.0,08 86018809,True,8.0
1,1,771155,joshua,elrick,23,andrea place,foxdown,east preston,2074,nsw,nsw,19120921.0,1912-09-21,34.0,34.0,1946.0,02 97793152,True,2.0
2,2,231932,alice,conboy,35,mountain circuit,NaN,prospect,2305,nsw,nsw,19810905.0,1981-09-05,22.0,22.0,2003.0,02 20403934,True,2.0
3,3,465838,sienna,craswell,39,cumberlegeicrescent,jodane,henty,3620,wa,wa,19840809.0,1984-08-09,30.0,30.0,2014.0,02 62832318,True,2.0
4,4,359178,joshua,bastiaans,144,lowrie street,NaN,campbell town,4051,nsw,nsw,19340430.0,1934-04-30,31.0,31.0,1965.0,03 69359594,True,3.0


Comme on a fait pour corriger les états, on peut directement utiliser un fuzzy matching sur les chaînes de caractères regroupant toutes les informations pertinentes de chaque row. Cependant, cette opération est en O(n^2).

C'est ce qu'on essaie ici, en supposant qu'une même personne peut avoir deux numéros de téléphone (donc on ne l'inclue pas dans la recherche de doublons), mais que les nom, prénom et adresse de quelqu'un identifient cette personne. On va donc inclure le nom, les deux adresses ainsi que le code postal dans ce processus.

In [ ]:
# Define better_scorer so that levenshtein package is not needed
def better_scorer(s1: str, s2: str) -> int:
    fuzz_ratio = fuzz.ratio(s1, s2)
    fuzz_ratio -= (s2.find(s1)*5)
    return fuzz_ratio

On teste une première fois sur une partie du dataset (les 1000 premières lignes) afin d'estimer le temps que prendra cette fonction pour s'exécuter.

In [155]:
df_patient["name_pc_info"] = df_patient.apply(
        lambda row: " ".join(str(info) for info in row[["given_name", "surname", "postcode", "street_number", "address_1", "address_2"]]).replace(" nan ", " "), axis=1)

start = time.time()
df_patient_name_pc_deduped = process.dedupe(df_patient.name_pc_info.iloc[:1000], scorer = better_scorer)
end = time.time()

print("Duration: ", end-start)

len([str(element) for element in df_patient_name_pc_deduped])

Duration:  18.309269905090332


819

On a donc pour 1000 lignes, environ 18 secondes d'exécution et on trouve 819 lignes si on enlève les doublons. Sachant qu'il y a encore 19000 lignes à parcourir, on risque de trouver beaucoup de doublons correspondant à ces 819 lignes restantes, elles vont donc encore diminuer.

18 secondes sachant que la complexité est en O(n^2), on peut estimer que pour le dataset complet, on prendra 400 fois plus de temps ( (20.000^2) / (1000^2) = 400), soit plus de deux heures sur l'ordinateur sur lequel je travaille (qui est loin d'être le plus performant).

*J'ai écrit le bout de code suivant la première fois que j'ai lancé process.dedupe sur l'entièreté des données, afin de ne pas avoir à le faire à chaque fois.*

In [132]:
# import pickle

# df_patient_name_pc_deduped = process.dedupe(df_patient.name_pc_info, scorer = better_scorer)
# to_save = [str(element) for element in df_patient_name_pc_deduped]
    
# pickle.dump(to_save, open("deduped.pkl", "wb+"))


In [156]:
deduped_patients = pickle.load(open("deduped.pkl", "rb"))

len(deduped_patients)

10283

On observe ici une réduction conséquente des données, de presque 50% (il reste 10283/19597 = 52% des données).

In [157]:
df_patient["is_original"] = df_patient.apply(lambda row: row.name_pc_info in deduped_patients, axis=1)

In [159]:
print(len(df_patient))
df_patient_no_duplicates = df_patient.drop_duplicates(subset="name_pc_info", inplace=False)

len(df_patient_no_duplicates[df_patient_no_duplicates.is_original])

19597


10283

Notre fonction detect_duplicates aura donc pour arguments le dataframe de données à dédupliquer, ainsi que la liste des features qu'on utilisera pour repérer et supprimer les doublons.

In [168]:
def detect_duplicates(df_patient: pd.DataFrame,
                     interesting_features: list = 
                      ["given_name", "surname", "postcode", "street_number", "address_1", "address_2"]) -> pd.DataFrame:
    
    df_patient["all_info"] = df_patient.apply(
        lambda row: " ".join(str(info) for info in row[interesting_features]).replace(" nan ", " "), axis=1)
    
    f = process.dedupe(df_patient.all_info, scorer = better_scorer)

    result_df = df_patient.drop_duplicates(subset="all_info")

    result_df["original"] = result_df.apply(lambda row: row.all_info in f, axis=1)
    result_df = result_df[result_df.original].drop(columns=["all_info", "original"])
    
    return result_df
    

In [161]:
start = time.time()
test = detect_duplicates(df_patient.iloc[:1000])
end=time.time()
print(end-start)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


19.20836901664734


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [162]:
test.shape

(819, 25)

On retrouve bien ici les 18 secondes et 819 clés uniques trouvées initialement.

In [ ]:
# To uncomment only right before running for the last time before pushing

# start = time.time()
# df_patient_no_duplicates = detect_duplicates(df_patient)
# end = time.time()
# print(end-start)

Cet algorithme prend plus de 8000 secondes, soit près de 2h30 pour 20000 lignes sur mon ordinateur. Cela reste acceptable pour ces données, mais on peut imaginer une base de données beaucoup plus grande (selon https://www.data.gouv.fr/fr/reuses/statistiques-de-tests-de-depistage-du-coronavirus-covid-19-en-france-par-region-et-par-departement/ plus de 9 millions de tests ont été réalisés à ce jour en France). Dans ce cas, une complexité en O(n^2) n'est pas envisageable.

Au lieu de cela, essayons d'utiliser le LSH, qui regroupe des éléments similaires dans une même classe avec une grande probabilité. Une fois les groupes formés, nous pourrons reprendre le fuzzy matching sur chaque sous-groupe. Le LSH réalise un hash où plus deux éléments (ici des chaînes de caractères) sont similaires, plus leur probabilité de collision est forte.

Cependant, nous risquons de passer à côté de certains doublons, puisqu'il arrive que des chaînes proches ne se retrouvent pas dans le même groupe avec le LSH. C'est pourquoi dans les arguments donnés en entrées de LSH, on donne plus de poids aux faux négatifs qu'aux faux positifs (qui seront de toute façon filtrés par le fuzzy matching plus tard): c'est l'argument 'weights' de la fonction detect_duplicates_lsh. On ne peut cependant pas mettre un poids trop fort, en risquant de se retrouver avec deux groupes avec la moitié des échantillons chacun par exemple, ce qui en effet minimise le risque de mettre deux éléments se ressemblant dans différents groupes, mais ne nous aide pas pour la phase de fuzzy matching en temps réduit.

In [104]:
def detect_duplicates_lsh(df_patient: pd.DataFrame,
                      interesting_features: list = ["given_name", "surname", "postcode", "state_corrected",
                                              "street_number", "address_1", "address_2", "suburb"],
                      jaccard_threshold: float=0.4, 
                      num_perm: int=128, 
                      weights: tuple=(0.1,0.9), 
                      nb_ngrams: int=3,
                      fuzzy_threshold: int=65) -> pd.DataFrame:
    
    # Create columns grouping all information of interest to deduplicate rows
    df_patient["all_info"] = df_patient.apply(
        lambda row: " ".join(str(info) for info in row[interesting_features]).replace(" nan ", " "). replace(" nat ", " "), axis=1)

    # Locality sensitive hashing
    lsh = datasketch.MinHashLSH(threshold=jaccard_threshold, num_perm=num_perm, weights=weights)

    minhashes = {}
    for c, i in enumerate(tqdm.tqdm(df_patient["all_info"])):
        minhash = datasketch.MinHash(num_perm=num_perm)
        for d in nltk.ngrams(i, nb_ngrams):
            minhash.update("".join(d).encode('utf-8'))
        lsh.insert(c, minhash)
        minhashes[c] = minhash
        
    df_patient["group_size"] = 0
    for i in tqdm.tqdm(range(len(df_patient))):
        df_patient.at[i, "group_size"] = len(lsh.query(minhashes[i]))
        
    # Find duplicates in groups found
    df_patient["processed"] = False 
    df_patient["original"] = False 
    df_patient["group"] = 0
    group = 0

    # Loop through all rows of dataframe
    for i in tqdm.tqdm(range(len(df_patient))):
        current_element = df_patient.iloc[i]

        # If the current element is not already part of a group
        if not current_element.processed:
            group +=1
            df_patient.at[i, "processed"] = True
            df_patient.at[i, "original"] = True
            df_patient.at[i, "group"] = group

            # Retrieve all elements colliding with the current one
            values_to_compare = df_patient.iloc[lsh.query(minhashes[i])]
            values_to_compare = values_to_compare[values_to_compare.processed == False].all_info
            # print(len(values_to_compare))

            # Extract distance of current element to every other colliding element
            result_comparison = process.extract(current_element.all_info, values_to_compare, scorer = better_scorer)
            
            string_match = [comp[0] for comp in result_comparison]
            list_match = [comp[0] for comp in result_comparison if comp[1]>fuzzy_threshold]

            # Mark matches as processed
            for m in list_match:
                df_patient.at[df_patient.loc[df_patient.all_info==m].index, "processed"] = True
                df_patient.at[df_patient.loc[df_patient.all_info==m].index, "group"] = True
                
    return(df_patient)
    
        

In [107]:
import time

start = time.time()
detect_duplicates_lsh(df_patient, fuzzy_threshold = 75, interesting_features = ["given_name", "surname", "postcode","address_1", "address_2", "suburb", "street_number"])
end = time.time()

print(end-start)
print(len(df_patient[df_patient.original])/len(df_patient))

100%|██████████| 19597/19597 [01:30<00:00, 217.71it/s]


213.531742811203
0.9193754146042762


In [12]:
df_patient.to_csv("australia_covid_no_dup.csv")

Cette méthode de calcul a l'avantage de réduire grandement le temps de calcul par rapport à la méthode précédente, ce qui nous permet de gérer une base de données potentiellement de plus en plus grande (si par exemple, il y a chaque jour des mises à jour des données avec de nouveaux patients et de nouveaux tests) sans GPU.

Cependant, le nombre de doublons trouvés n'est pas comparable à la quantité de doublons trouvés par fuzzywuzzy uniquement. Il semblerait que la probabilité de collision ne soit pas assez grande pour se contenter de ne regarder que dans le groupe de chaque valeur. De plus, il y a beaucoup plus de paramètres à fixer qui peuvent modifier l'output (en diminuant *fuzzy_treshold*, on va par exemple avoir beaucoup plus de duplicates; par exemple, si l'on augmente le nombre de features utilisées, il faut descendre *fuzzy_threshold* en conséquence puisque la similarité peut baisser avec la longueur de la chaîne de caractères). Cela étant, cette fonction reste intéressante pour des bases de données plus conséquentes, où le fuzzy matching est trop coûteux en temps, afin de repérer une partie au moins des doublons.

In [198]:
# Tests pytest

TEST_INPUT = pd.DataFrame({"given_name": ["jane", "jane", "john", "jon"],
                               "surname": ["doe", "dooe", "doe", "doe"],
                               "address_1": ["willstreet", "will street", "abbey road", "abbey"],
                               "address_2": ["", "", "", "road"],
                               "street_number": ["1", "1", "2", ""],
                               "postcode": ["1234", "12", "456", "456"]})

EXPECTED_RESULT = pd.DataFrame({"given_name": ["jane", "john"],
                               "surname": ["doe", "doe"],
                               "address_1": ["willstreet", "abbey road"],
                               "address_2": ["", ""],
                               "street_number": ["1", "2"],
                               "postcode": ["1234", "456"]})

def test_detect_duplicates() -> None:
    result_to_test = detect_duplicates(TEST_INPUT)
    print(result_to_test)
    
    for row in result_to_test:
        assert (row in EXPECTED_RESULT)
        
    assert TEST_INPUT.shape[0]>=result_to_test.shape[0]
    
test_detect_duplicates()

  given_name surname   address_1 address_2 street_number postcode
0       jane     doe  willstreet                       1     1234
2       john     doe  abbey road                       2      456



- Il serait intéressant, lorsque plusieurs lignes sont désignées comme doublons, de prendre pour chaque colonne la valeur la plus présente dans le groupe de doublons. Avec fuzzywuzzy, c'est la valeur la plus longues qui est sélectionnée, alors que "jaane" devrait être "jane" par exemple. Cela pourrait aussi permettre de remplir les données manquantes.

- Quand on cherche les doublons dans `detect_duplicates`, il serait intéressant d'y inclure df_pcr dans la liste des arguments; ainsi, on pourrait sélectionner, dans un groupe de doublons, celui dont le patient_id est inclus dans df_pcr, afin de maximiser les données analysables.